## Assignment-07 First Step of using machine learning and models.

In [29]:
import os
import re
import scipy
import numpy as np
import pandas as pd
from pyltp import Segmentor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [2]:
fname = "D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\Week_05_0803_pyltp\\sqlResult_1558435.csv"

In [3]:
data = pd.read_csv(fname, encoding='gb18030')

In [4]:
data.head(1)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm


In [5]:
len(data)

89611

In [6]:
xinhua_news = data[data.source == '新华社']

In [7]:
len(xinhua_news) / len(data)

0.8778051801676133

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1566105348906&di=ee9a2de91207767364853d4decc6cca3&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_png%2FTicO2kbP6Ao5sCsSQDpehZiczLdC6hDCNvoicjcOCEKX2bLxBc9gVOw28zHyFibfIWq9ceRibP6HDTKReGkr6YyTfQQ%2F640%3Fwx_fmt%3Dpng)

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [8]:
label = (data.source == '新华社').astype(int)
content = data.content

In [9]:
data_new = pd.DataFrame({'content':content, 'label':label})

In [10]:
data_new.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


### Step3: 使用tfidf进行文本向量化

#### 新闻内容切词整理 

In [10]:
ltp_model_path = "D:\\Github\\NLP\\Projects\\data\\ltp_data_v3.4.0"
segmentor_model_path = os.path.join(ltp_model_path, 'cws.model')
segmentor = Segmentor()
segmentor.load(segmentor_model_path)

In [11]:
def ltp_cut(document):
    result = ''
    if isinstance(document, str):
        words = segmentor.segment(document)
        words = [re.findall('\d*\w+', x) for x in words]
        result = ' '.join([x[0] for x in words if x and x[0] !='n'])  # pyltp会把换行符\n给分开
        return result if result else None

#### 切词保存数据，这一步还是要花几分钟的

In [16]:
data_new['content_cut'] = data_new.content.apply(lambda i: ltp_cut(i))
data_new.to_csv('cleaned_data.csv')

In [2]:
# index_col 参数不可省略，否则会出现未命名列
df = pd.read_csv('cleaned_data.csv', index_col=0)

In [6]:
# 删除 nan 无数据行
df.dropna(inplace=True)

In [7]:
len(df)

86863

In [15]:
df.tail()

,content,label,content_cut
89606,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
89607,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
89608,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
89609,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
89610,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


In [16]:
# 重置索引为连续
df.reset_index(drop=True, inplace=True)

In [17]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


In [18]:
len(df)

86863

#### 文本向量化

In [19]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=200)

In [20]:
X = vectorized.fit_transform(df.content_cut)

In [21]:
X.shape

(86863, 200)

In [66]:
df.label.value_counts()

1    78472
0     8391
Name: label, dtype: int64

* scipy库中consine的实现为：$$cosine(u, v) = 1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$  取值范围[0, 2];因此这个函数的结果越小说明越相似。

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. [reference](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [42]:
X = X.toarray()

In [11]:
y = df.label

In [44]:
y.shape, X.shape

((86863,), (86863, 200))

In [49]:
Xtrain,Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
Xtrain, Xcv, ytrain, ycv = train_test_split(Xtrain, ytrain, test_size=.25)
Xtrain.shape,ytrain.shape, Xcv.shape, ycv.shape, Xtest.shape , ytest.shape

((52117, 200), (52117,), (17373, 200), (17373,), (17373, 200), (17373,))

+ 参照scikit learning的示例，构建你的第一个KNN机器学习模型。[reference](https://scikit-learn.org/stable/modules/neighbors.html#neighbors)

### Step5-6: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

#### KNN
慢。

In [50]:
clf = KNC()
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.92672537846083

In [51]:
ypred = clf.predict(Xtest)

In [52]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1651
           1       0.95      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.80      0.75      0.77     17373
weighted avg       0.92      0.93      0.92     17373



非新华社文章的判准率很低。

#### LogisticRegression
快。

In [72]:
logi_clf = LogisticRegression(solver='lbfgs')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9717953145685835, 0.9709894664134001)

In [70]:
ypred_logi = logi_clf.predict(Xtest)
print(classification_report(ytest, ypred_logi))

              precision    recall  f1-score   support

           0       0.91      0.78      0.84      1651
           1       0.98      0.99      0.98     15722

    accuracy                           0.97     17373
   macro avg       0.94      0.88      0.91     17373
weighted avg       0.97      0.97      0.97     17373



In [73]:
ypred_logi = logi_clf.predict(Xcv)
print(classification_report(ycv, ypred_logi))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84      1654
           1       0.98      0.99      0.98     15719

    accuracy                           0.97     17373
   macro avg       0.94      0.89      0.91     17373
weighted avg       0.97      0.97      0.97     17373



表现中等，作备选，如果KNN表现不如意的话。

### Step7: 调整不同的参数，观察变化

#### 默认参数

In [50]:
clf = KNC()
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.92672537846083

In [51]:
ypred = clf.predict(Xtest)

In [52]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1651
           1       0.95      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.80      0.75      0.77     17373
weighted avg       0.92      0.93      0.92     17373



#### 调整weights

In [92]:
clf = KNC(weights='distance')
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.9320209520520347

In [93]:
ypred = clf.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.68      0.57      0.62      1651
           1       0.96      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.82      0.77      0.79     17373
weighted avg       0.93      0.93      0.93     17373



非新华社文章的判准率还是很低，比一般LogisticRegression表现都差。

#### 调整Tfidf的文章向量数，200 --> 800

In [4]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=800)

In [8]:
X = vectorized.fit_transform(df.content_cut)

In [9]:
X.shape

(86863, 800)

In [12]:
X2 = X.toarray()
X2.shape, y.shape

((86863, 800), (86863,))

In [13]:
def corpus_split(X, y):
    Xtrain,Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
    Xtrain, Xcv, ytrain, ycv = train_test_split(Xtrain, ytrain, test_size=.25)
    return Xtrain,ytrain, Xcv, ycv, Xtest , ytest

In [14]:
Xtrain, ytrain, Xcv, ycv, Xtest, ytest = corpus_split(X2, y)
Xtrain.shape,ytrain.shape, Xcv.shape, ycv.shape, Xtest.shape , ytest.shape

((52117, 800), (52117,), (17373, 800), (17373,), (17373, 800), (17373,))

In [15]:
clf = KNC(weights='distance')
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.9118747481724515

In [16]:
ypred = clf.predict(Xtest)

In [17]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.53      0.68      0.60      1699
           1       0.96      0.94      0.95     15674

    accuracy                           0.91     17373
   macro avg       0.75      0.81      0.77     17373
weighted avg       0.92      0.91      0.92     17373



KNN实在是太慢了，打算用Logistics进行参数调优。

#### LogisticRegression

In [26]:
logi_clf = LogisticRegression(solver='lbfgs')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9711045875784263, 0.9707016635008346)

In [27]:
ypred_logi = logi_clf.predict(Xtest)

In [28]:
print(classification_report(ytest, ypred_logi))

              precision    recall  f1-score   support

           0       0.93      0.76      0.84      1699
           1       0.97      0.99      0.98     15674

    accuracy                           0.97     17373
   macro avg       0.95      0.88      0.91     17373
weighted avg       0.97      0.97      0.97     17373



文本向量大小貌似对结果影响不大。

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？

In [31]:
params = {'C':[.01, .03, .1, .3, 1, 3, 10], 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
clf_logi = LogisticRegression(solver='lbfgs')
clf = GridSearchCV(clf_logi, params, cv=5)
clf.fit(Xtrain, ytrain)

C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs 

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [40]:
pd.DataFrame(clf.cv_results_).tail()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
30,12.122041,0.487327,0.019001,0.000633,10,newton-cg,"{'C': 10, 'solver': 'newton-cg'}",0.980622,0.983404,0.984459,0.981579,0.981002,0.982213,0.001474,1
31,4.806591,0.156656,0.017799,0.000749,10,lbfgs,"{'C': 10, 'solver': 'lbfgs'}",0.980622,0.983404,0.984459,0.981483,0.981002,0.982194,0.001483,5
32,1.363001,0.013709,0.017002,0.000631,10,liblinear,"{'C': 10, 'solver': 'liblinear'}",0.980526,0.983404,0.984459,0.981675,0.981002,0.982213,0.001488,1
33,14.128601,1.809070,0.016202,0.000399,10,sag,"{'C': 10, 'solver': 'sag'}",0.980526,0.983404,0.984459,0.981675,0.981002,0.982213,0.001488,1
34,22.189801,0.604848,0.016202,0.000746,10,saga,"{'C': 10, 'solver': 'saga'}",0.980622,0.983404,0.984459,0.981579,0.981002,0.982213,0.001474,1


In [35]:
clf.best_params_, clf.best_score_

({'C': 10, 'solver': 'newton-cg'}, 0.9822130974538058)

In [37]:
logi_clf = LogisticRegression(C=10, solver='newton-cg')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9831923099061762, 0.9826167040810453)

In [38]:
ypred = logi_clf.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1699
           1       0.99      0.99      0.99     15674

    accuracy                           0.98     17373
   macro avg       0.96      0.94      0.95     17373
weighted avg       0.98      0.98      0.98     17373



In [ ]:
ypred==ytest

非新华社文章的判准率还行。

### Step9: 找出所有预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。

In [43]:
X2.shape
# logi_clf.predict()

(86863, 800)

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。

In [42]:
svc = SVC(gamma='auto')
svc.fit(Xtrain, ytrain)
svc.score(Xcv, ycv)

0.9011109192425028

In [44]:
ypred = svc.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1699
           1       0.90      1.00      0.95     15674

    accuracy                           0.90     17373
   macro avg       0.45      0.50      0.47     17373
weighted avg       0.81      0.90      0.86     17373



C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
